# Trusted Context Lab

Prepare for lab:

In [ ]:
%run Lab_macros.ipynb

Execute the following commands as instance owner db2inst1 in a terminal window.
But first you have to replace XXX.XXX.XXX.XXX with the IPV4 address of your system. (Use ifconfig in the terminal window to get the IP address. Don't select the IP of the local loopback device, use your regular Ethernet interface. The IPV4 address is prefixed by "inet".)

db2 catalog tcpip node n1 remote XXX.XXX.XXX.XXX server 50000

db2 catalog database sample as samplecl at node n1

db2 terminate

The reason why we do this is that trusted contexts can only be used with a TCP/IP connection from a remote system (client or other server). To avoid a second virtual machine we simulate this by connection via the database sample1 which we just cataloged.

Depending on what other sublabs have already been executed, either the user db2inst1 or the user alex may have SECADM authority. Therefore, let's find out who has SECADM authority on the database sample:

In [ ]:
%sql CONNECT TO sample USER db2inst1 USING db2inst1

In [ ]:
%%sql
SELECT 'DB2INST1', authority, d_user  
FROM table(auth_list_authorities_for_authid('DB2INST1','U')) 
WHERE authority = 'SECADM';

In [ ]:
%%sql
SELECT 'ALEX', authority, d_user  
FROM table(auth_list_authorities_for_authid('ALEX','U')) 
WHERE authority = 'SECADM';

The user for whom D_USER is set to "Y", is the security admin. We have to use the corresponding user later when we have to connect as security admin.

The password for db2inst1 is db2inst1.

The password for alex is p123.

Now let us create tables with two different users (peter and paul) and insert some rows. These users themselves have already been created in the previous lab.

In [ ]:
%sql CONNECT TO sample USER peter USING p123

In [ ]:
%%sql
CREATE TABLE t1 (c0 INT, c1 CHAR(10));

In [ ]:
%%sql
INSERT INTO t1 VALUES (0,'peter');
INSERT INTO t1 VALUES (1,'peter');

In [ ]:
%%sql
SELECT substr(authid,1,20) AS authid
    , authidtype
    , privilege
    , grantable
    , substr(objectschema,1,12) AS objectschema
    , substr(objectname,1,30) AS objectname
    , objecttype 
FROM sysibmadm.privileges
WHERE objectname = 'T1'

In [ ]:
%sql CONNECT TO sample USER paul USING p123

In [ ]:
%%sql
CREATE TABLE t2 (c0 INT, c1 CHAR(10));

In [ ]:
%%sql
INSERT INTO t2 VALUES (0,'paul');
INSERT INTO t2 VALUES (1,'paul');

In [ ]:
%%sql
SELECT substr(authid,1,20) AS authid
    , authidtype
    , privilege
    , grantable
    , substr(objectschema,1,12) AS objectschema
    , substr(objectname,1,30) AS objectname
    , objecttype 
FROM sysibmadm.privileges
WHERE objectname = 'T2'

# Reading from the tables

Let's check who can access which table

In [ ]:
%sql CONNECT TO sample USER peter USING p123

In [ ]:
%%sql
SELECT * FROM peter.t1;

In [ ]:
%sql CONNECT TO sample USER paul USING p123

In [ ]:
%%sql
SELECT * FROM peter.t1;

In [ ]:
%%sql
SELECT * FROM paul.t2;

# We expect to see the same when connecting via sample1

Let's check it.

In [ ]:
%sql CONNECT TO samplecl USER peter USING p123

In [ ]:
%%sql
SELECT * FROM peter.t1;

In [ ]:
%sql CONNECT TO samplecl USER paul USING p123

In [ ]:
%%sql
SELECT * FROM peter.t1;

In [ ]:
%%sql
SELECT * FROM paul.t2;

# Use of trusted context to give paul access to peter's table in a specific context

The user creating the Trusted Context has to have SECADM authority.

Therefore, replace in the statement below secadmuser and secadmpassword with the user who has this authority and the respective password. (You had determined this at the beginning of this lab.)

In [ ]:
%sql CONNECT TO sample USER secadmuser USING secadmpassword

We will set a default role for user paul in the trusted context. This role should give paul access to peter's table.

First we create the role:

In [ ]:
%%sql
CREATE ROLE role_peter;

Next we grant SELECT permission on peter's table to this role:

In [ ]:
%sql GRANT SELECT ON peter.t1 TO role_peter

Now we are ready to define the trusted context.
    
We want to allow give user paul the permissions associated with the role role_peter when paul is connecting from a specific system to the Db2 server. For the purposes of this lab this system will be just our virtual machine. Therefore, we have to get our IPV4 address (how to get this address is described at the beginning of this notebook.) and replace the XXX.XXX.XXX.XXX in the cell below by this IP address. 

In [ ]:
%%sql
CREATE TRUSTED CONTEXT tc BASED UPON CONNECTION 
USING SYSTEM AUTHID paul 
ATTRIBUTES (ADDRESS 'XXX.XXX.XXX.XXX') 
DEFAULT ROLE role_peter ENABLE;

Now we are ready to test the trusted context. For this we connect as user paul to the database samplecl (which is our remote client for the purposes of this lab)

In [ ]:
%sql CONNECT TO samplecl USER paul USING p123

In [ ]:
%%sql
SELECT * FROM peter.t1;

We were now able to read peter.t1 because of the default role specified in the trusted context.

In [ ]:
%sql CONNECT TO sample USER paul USING p123

In [ ]:
%%sql
SELECT * FROM peter.t1;

When connecting locally to the database on which the trusted context was defined, we don't have the permission to access the table.

# Disabling and Enabling Trusted Contexts

We had created our trusted context already in the enabled state. But trusted contexts can be enabled and disabled. Let's check it.

The user enabling and disabling the Trusted Context has to have SECADM authority.

Therefore, replace in the statement below secadmuser and secadmpassword with the user who has this authority and the respective password. (You had determined this at the beginning of this lab.)

In [ ]:
%sql CONNECT TO sample USER secadmuser USING secadmpassword

In [ ]:
%sql ALTER TRUSTED CONTEXT tc ALTER DISABLE

In [ ]:
%sql CONNECT TO samplecl USER paul USING p123

In [ ]:
%%sql
SELECT * FROM peter.t1;

Now connect again with the user who has SECADM authority.

Therefore, replace in the statement below secadmuser and secadmpassword with the user who has this authority and the respective password. (You had determined this at the beginning of this lab.)

In [ ]:
%sql CONNECT TO sample USER secadmuser USING secadmpassword

In [ ]:
%sql ALTER TRUSTED CONTEXT tc ALTER ENABLE

In [ ]:
%sql CONNECT TO samplecl USER paul USING p123

In [ ]:
%%sql
SELECT * FROM peter.t1;

# How is a trusted context stored?

In [ ]:
%sql CONNECT TO sample USER db2inst1 USING db2inst1

In [ ]:
%%sql
SELECT * FROM syscat.contexts;

The entry with contextname "TC" corresponds to the trusted context we created. Which of the information did we specify explicitly, which was created implicitly?

In [ ]:
%%sql
SELECT * FROM syscat.contextattributes;

Why are there two rows for our trusted context in this table?

In [ ]:
%%sql
SELECT * FROM syscat.surrogateauthids;

There are no entries for the trusted context we created in the surrogateauthids table. We would have had specify users in the trusted context to which we might want to switch. Check the documentation (in IBM Knowledge Center) for trusted context on this and think how a example using this functionality could look like.

You are done with the trusted context lab now.

# Clean up

If you have messed up this trusted context lab, you can clean up all objects in the database and start from the top of this notebook.

In [ ]:
%sql CONNECT TO sample USER db2inst1 USING db2inst1

In [ ]:
%sql DROP TABLE peter.t1

In [ ]:
%sql DROP TABLE paul.t2

The user dropping roles and trusted contexts has to have SECADM authority.

Therefore, replace in the statement below secadmuser and secadmpassword with the user who has this authority and the respective password. (You had determined this at the beginning of this lab.)

In [ ]:
%sql CONNECT TO sample USER secadmuser USING secadmpassword

In [ ]:
%sql DROP TRUSTED CONTEXT tc

In [ ]:
%sql DROP ROLE role_peter